In [459]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [460]:
# read words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [461]:
len(words)

32033

In [462]:
# build char/int mappings
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [463]:
# build dataset

block_size = 3 # context length; how many chars do we take as input when predicting the next char?
X, Y = [], []
for w in words:

    # print(w)
    context = [0 for i in range(block_size)]
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # sliding window of size 3 along the word

X = torch.tensor(X)
Y = torch.tensor(Y)

In [464]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [465]:
# embeddings

# each char will be represented by an embedding vector in some n-dimensional space
# the "location" of each char and its distance/direction from other chars will kinda tell us what it is
# we will start with a 2-dimensional embedding vector
C = torch.randn((27, 2)) # 27 chars, 2D vector

In [466]:
# embedding a single integer

# we can just index
C[5]

# we can also use one-hot encoding and matrix multiplication!
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.2639,  0.9414])

In [467]:
emb = C[X]
emb.shape
# the shape is [32, 3, 2]
# -> 32 data samples
# -> 3 inputs (3 chars for context)
# -> 2 embeddings per input

torch.Size([228146, 3, 2])

In [468]:
# embeddings are 2-dim, and we have 3, so 6 embeddings as input
# number of neurons in second layer is up to us
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [469]:
# we want to multiply emb @ W1 to get the weighted sum
# currently can't multiply since dims don't match
# idea: flatten emb from [32, 3, 2] to [32, 6]

In [470]:
# we can grab all entries with dim1 = 0
# this is equivalent to all the embeddings for the first char
emb[:, 0, :]

tensor([[ 0.2698,  0.3137],
        [ 0.2698,  0.3137],
        [ 0.2698,  0.3137],
        ...,
        [-0.8662,  1.2053],
        [-0.8662,  1.2053],
        [-0.4417,  0.2726]])

In [471]:
# we have 3 [32, 2] tensors and want to cat them into a single [32, 6] tensor
# dim tells us which dimension should be EXTENDED in the concatenation
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1)

tensor([[ 0.2698,  0.3137,  0.2698,  0.3137,  0.2698,  0.3137],
        [ 0.2698,  0.3137,  0.2698,  0.3137, -0.2639,  0.9414],
        [ 0.2698,  0.3137, -0.2639,  0.9414,  0.3559,  1.4330],
        ...,
        [-0.8662,  1.2053, -0.8662,  1.2053, -0.4417,  0.2726],
        [-0.8662,  1.2053, -0.4417,  0.2726, -0.8662,  1.2053],
        [-0.4417,  0.2726, -0.8662,  1.2053, -1.4335, -0.1182]])

In [472]:
# we can use torch.unbind() to remove a dim from a tensor
torch.unbind(emb, 1)
# this is equivalent to [emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]]

(tensor([[ 0.2698,  0.3137],
         [ 0.2698,  0.3137],
         [ 0.2698,  0.3137],
         ...,
         [-0.8662,  1.2053],
         [-0.8662,  1.2053],
         [-0.4417,  0.2726]]),
 tensor([[ 0.2698,  0.3137],
         [ 0.2698,  0.3137],
         [-0.2639,  0.9414],
         ...,
         [-0.8662,  1.2053],
         [-0.4417,  0.2726],
         [-0.8662,  1.2053]]),
 tensor([[ 0.2698,  0.3137],
         [-0.2639,  0.9414],
         [ 0.3559,  1.4330],
         ...,
         [-0.4417,  0.2726],
         [-0.8662,  1.2053],
         [-1.4335, -0.1182]]))

In [473]:
# using cat and unbicnd together
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([228146, 6])

In [474]:
# THERE IS A BETTER WAY

# let's take a tensor with elements from 0 to 17
a = torch.arange(18)

# it's currently just a [18] tensor
# calling a.storage shows that pytorch stores all tensors as 1-dim lists
a.storage()

# pytorch allows us to "view" this 1-dim list with any dimensions, as long as the product equals the number of elements
print(a.view(2, 9))
print(a.view(2, 3, 3))

# pytorch can also infer a dim based on the others dims by using -1
print(a.view(-1, 9))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])
tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])


In [475]:
emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

RuntimeError: shape '[32, 6]' is invalid for input of size 1368876

In [476]:
# we can multiply now!!!
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h.shape

torch.Size([228146, 100])

In [477]:
# 100 neurons in last layer
# 27 outputs (1 for each char)
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [478]:
logits = h @ W2 + b2

In [479]:
logits.shape

torch.Size([228146, 27])

In [480]:
counts = logits.exp()

In [481]:
probs = counts / counts.sum(1, keepdim=True)

In [482]:
# now pluck out the row in prob corresponding to the elements in Y
probs[torch.arange(X.shape[0]), Y]

tensor([1.3445e-08, 6.7046e-10, 2.9847e-13,  ..., 8.4717e-07, 5.1055e-06,
        4.4023e-12])

In [483]:
# get NLL
loss = -probs[torch.arange(probs.shape[0]), Y].log().mean()

In [484]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(Y.shape[0]), Y].log().mean()
loss

tensor(17.9031)

In [485]:
# cross entropy calculates the loss from the logits much more efficiently!
F.cross_entropy(logits, Y)

tensor(17.9031)

In [486]:
# cross_entropy is better because

# 1. it uses less memory and more efficient calculations (something about fused kernels)

# 2. do math better (uses analytical derivatives for backward pass, etc.)

# 3. it is able to handle very positive numbers

logits = torch.tensor([-5, -3, 0, 100])
counts = logits.exp()
# this produces an inf when we evaluate exp(100)
# but very negative numbers are fine, since exp(-inf) just goes to 0, which isn't a problem
logits = torch.tensor([-5, -3, 0, -100])
counts = logits.exp()

# note that if we offset logits by some constant x, probs will remain the same
# this is because when we do logits.exp(), all the elements have been multiplied by exp(x)
# so when we normalize them nothing changes

logits1 = torch.tensor([-5, -3, 0, -100])
counts1 = logits.exp()
probs1 = counts1 / counts1.sum(0)
logits2 = torch.tensor([-5, -3, 0, -100]) + 20
counts2 = logits.exp()
probs2 = counts2 / counts2.sum(0)

probs1 == probs2 # produces True

# to solve the problem, pytorch internally uses this and offsets logits so that the greatest element is 0

tensor([True, True, True, True])

In [487]:
# ----------------- COMBINING THE CODE WE JUST WROTE ----------------------

In [488]:
# training split, dev/validation split, test split
# 80%, 10%, 10%
# train parameters (weights & biases), train hyperparameters (things we get to choose, such as size of embeddings), evaluation

In [516]:
# build dataset

def build_dataset(words):
    block_size = 3 # context length; how many chars do we take as input when predicting the next char?
    X, Y = [], []
    for w in words:
    
        # print(w)
        context = [0 for i in range(block_size)]
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # sliding window of size 3 along the word
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words)) # end of training split
n2 = int(0.9 * len(words)) # end of dev split

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182441, 3]) torch.Size([182441])
torch.Size([22902, 3]) torch.Size([22902])
torch.Size([22803, 3]) torch.Size([22803])


In [517]:
Xtr.shape, Ytr.shape

(torch.Size([182441, 3]), torch.Size([182441]))

In [518]:
# initializing NN layers and parameters with random values
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 20), generator=g)
W1 = torch.randn((60, 500), generator=g)
b1 = torch.randn(500, generator=g)
W2 = torch.randn((500, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [519]:
sum(p.nelement() for p in parameters) # total number of params

44567

In [520]:
for p in parameters:
    p.requires_grad = True

In [508]:
# optimizing learning rate

# we can basically try a bunch of different learning rates and see how the loss is impacted
lre = torch.linspace(-3, 0, 1000)
lrs = 10 ** lre
# this creates a linearly spaced tensor with numbers from 10**-3 to 10**0 (0.001 to 1)

In [521]:
lri = []
lossi = []
stepi = []

In [523]:
for i in range(300000):

    # construct random minibatch
    ix = torch.randint(0, Xtr.shape[0], (32,))

    # forward pass
    emb = C[Xtr[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 60) @ W1 + b1)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Ytr[ix])
    # print(loss.item())
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    # lr = lrs[i]
    if i < 100000:
        lr = 0.1
    elif i < 200000:
        lr = 0.01
    else:
        lr = 0.001
    # lr = 0.1 if i < 100000 else 0.01 # for the last few steps, we can use a decay learning rate that is much smaller for fine-tuning
    for p in parameters:
        p.data += -lr * p.grad

    # track learning rate stats
    # lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())

In [ ]:
plt.plot(stepi, lossi)

In [526]:
emb = C[Xtr] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 60) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.0098, grad_fn=<NllLossBackward0>)

In [527]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 60) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1646, grad_fn=<NllLossBackward0>)

In [ ]:
# loss for training set and dev set are very similar, so we're not overfitting
# we are actually underfitting, so we can increase the number of parameters in the NN
# easiest way is to increase the number of neurons in the hidden layer

In [458]:
g = torch.Generator().manual_seed(2147483647 + 1)

for _ in range(20):
    out = []
    context = [0 for _ in range(block_size)]
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

camrienor.
slyn.
emon.
arrianiel.
deck.
ali.
ustia.
alboxal.
jara.
caylon.
delynn.
neyd.
ett.
zuhelda.
aatizeracelynn.
torevonnial.
ann.
kaa.
juon.
issietten.


In [ ]:
# visualization of 2D embeddings

plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')

In [ ]:
plt.plot(lri, lossi)
# this graph has a min around -1.0, so 10**-1 == 0.1 is a fairly good learning rate